# Importing the libraries

In [2]:
#importing the libraries:
from  bs4  import BeautifulSoup
from urllib.request import Request,urlopen
import nltk
nltk.download('punkt')
import requests 
import re
import pandas as pd
import numpy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Creating function to extract Q&A from the website

In [3]:
# function to extract question and answer from the provided url.
#Input to this fuction are url,tag and class:
def qustn_answr_extract(link,tag,g_class):
  q_list=[]
  a_list=[]
  url=link
  req=Request(url,headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
            'AppleWebKit/537.36 (KHTML, like Gecko) '\
            'Chrome/75.0.3770.80 Safari/537.36'})

  page = urlopen(req).read()
  soup = BeautifulSoup(page, 'html.parser')
  sent = list(soup.find_all(tag,class_ =g_class))
 
  rx =r"\.(?=\S)"
  rx1=r"\?(?=\S)"
  

  para=[' '.join(list(map(lambda x:x.text.strip().replace('\n',' '),sent)))]

  for text in para:
      q_sent = nltk.sent_tokenize(text)
      a_sent=q_sent.copy()
      i=0
      z=0
      while i<len(q_sent): #Extrating question:
          temp1_list=[]
          q_sent[i]=re.sub(rx,". ",q_sent[i])
          q_sent[i]=re.sub(rx1,"? ",q_sent[i])
          if q_sent[i][-1]=='?':
              temp1_list.append(q_sent[i])
              j=i+1
              while j<len(q_sent):
                  if q_sent[j][-1]=='?':
                      temp1_list.append(q_sent[j])
                      q_sent.remove(q_sent[j])
                  else:
                      break
                  
              q_list.append(' '.join(temp1_list)) 
          i+=1 
          
      while z<len(a_sent): #Extracting Answers:
          temp2_list=[]
          a_sent[z]=re.sub(rx,". ",a_sent[z])
          a_sent[z]=re.sub(rx1,"? ",a_sent[z])
          if a_sent[z][-1]=='.' and a_sent[z-1][-1]=='?':
              temp2_list.append(a_sent[z])
              j=z+1
              while j<len(a_sent):
                  if a_sent[j][-1]=='.':
                      temp2_list.append(a_sent[j])
                      a_sent.remove(a_sent[j])
                  else:
                      break
              a_list.append(' '.join(temp2_list))
          z+=1      
  
  return q_list, a_list

In [4]:
#question and answer extraction using the above function from mirraw.com
questn_ans=qustn_answr_extract('https://www.mirraw.com/pages/faq','div','faq_qas_block') #calling above function with input


In [5]:
#question and answer extraction using the above function from koovs.com

qustn_answr_extract('https://www.koovs.com/info/faq#whyReg', 'div','faqBrief')


(['To shop at KOOVS. COM do I have to register? If yes, why?',
  'How do I register myself?',
  'Do I need to give my email ID and mobile number for registration?',
  'Is my personal information secure at KOOVS. COM?',
  'Do I need to pay for registration?',
  'For how long is my registration valid?',
  'I forgot my password, what do I do?',
  'If I need to update my account information, what steps do I follow?',
  'How do I unsubscribe from your promotional Emails?',
  'Is it possible?',
  'Which category products do you sell online?',
  'How do I know my size?',
  'Do you provide warranty/guarantee for merchandise?',
  'Why is that so?',
  'How do I add more products to my order?',
  'How can I use a voucher in the ‘My Account’ section to place an order?',
  'What are the shipping charges I need to pay for an order under INR 1199?',
  'Is there a limit to the quantity that I can order?',
  'How do I apply the refund voucher provided to me?',
  'Do you take orders over phone?',
  'Wha

In [6]:
#question and answer extraction using the above function from anuradhaartjewellery.com
qustn_answr_extract('https://www.anuradhaartjewellery.com/faq', 'div','card')


(['How can I return a product?',
  'What are the steps for order cancellation?',
  'In how many days my refund will be initiated?',
  'What is the mode of a refund?',
  'NEFT Transfer: For NEFT transfer you have to provide NEFT details so that refund will be initiated Is there a return option available for international orders?',
  'How a domestic user can make the payment? There is two payment mode available at Anuradha Art Jewellery:                                        Cash on delivery: You can make the payment to delivery boy at the time of product delivery                                        Online Payment: You can pay online using Credit card/ Debit card/ Net banking/ Cash cards How International user can make the payment? There are two payment mode available for international orders:                                        Payment through Credit Card: You can pay online using credit cards                                        Paypal: You can make payment through paypal What

In [7]:
#question and answer extraction using the above function from luluandsky.com and streetstylestore.com

questn_ans_1=qustn_answr_extract('https://www.luluandsky.com/faqs','div','panel-sec')
questn_ans_2=qustn_answr_extract('https://streetstylestore.com/index.php?id_cms=18&controller=cms','div','container-fluide')

In [9]:
#merging the retured question list from luluandsky.com and streetstylestore.com
#merging the retured answer list from luluandsky.com and streetstylestore.com
questn=[]
answer=[]
for i in questn_ans_1[0][:len(questn_ans_1[1])]+questn_ans_2[0][:len(questn_ans_2[1])]:
  questn.append(i)
for i in questn_ans_1[1][:len(questn_ans_1[0])]+questn_ans_2[1][:len(questn_ans_2[0])]:
  answer.append(i)

In [10]:
#making dictionary of the question and answers

Dic = {"Questions": questn, "Answers": answer}

#passing the dictionary data to the dataframe.

scrap_data = pd.DataFrame(Dic)
scrap_data
#exporting data in CSV.
scrap_data.to_csv("df.csv")

In [12]:
#importing the data containing extracted question and answer from the www.luluandsky.com website  
df = pd.read_csv("/content/df.csv")
df= df.drop(['Unnamed: 0'],1)

# Data cleaning

In [13]:
#creating function to clean the sentences 
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords


def clean_sentence(sentence, stopwords=False):
  
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  
  
  if stopwords:
    sentence = remove_stopwords(sentence)
    
  return sentence

def get_cleaned_sentences(df,stopwords=False):
  sents=df [["Questions"]];
  cleaned_sentences=[]
  
  for index,row in df.iterrows():
    cleaned=clean_sentence(row["Questions"],stopwords);
    cleaned_sentences.append(cleaned);
  return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df, stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

['register shop luluandsky com yes register', 'disclose email phone number registration', 'personal information secure luluandsky com', 'charges registration charges registration luluandskycom registration expire', 'new password', 'edit personal information', 'unsubscribe services', 'possible', 'touch luluandsky com', 'type products sell online', 'know size', 'payment', 'happens creditdebit card compromised making payment online', 'happens merchandise ordered stock', 'limit quantity order', 'cancel order', 'change shipping address order', 'return product', 'provide warrantyguarantee merchandise', 'deliver india', 'available deliver merchandise', 'order received damaged condition', 'book order delivery address', 'shipping charges andor additional charges', 'track order', 'stipulated delivery time order', 'receive different item ordered', 'dispatch order', 'product picked', 'return products ordered', 'refund', 'detect fraudulent emailscalls seeking sensitive personal confidential informa

In [14]:
#creating function which finds the cosine_similarity between the user query and trained data
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
  max_sim=-1;
  index_sim=-1;
  for index, faq_embedding in enumerate(sentence_embeddings):
  
    sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
    print(index, sim, sentences [index])
    if sim>max_sim:
      max_sim=sim;
      index_sim=index;
  print("\n")
  print("Question: ", question)
  print("\n");
  print("Retrieved: " ,FAQdf.iloc[index_sim, 0])
  print(FAQdf.iloc[index_sim,1])

#retrieveAndPrintFAQAnswer (question bedding,bow corpus,df,sentences);

# Word2vec algorithm- Skip-gram model

In [15]:
#Use gensim to load a word2vec model,havinig embedding of 300 dimensions, pretrained on google news
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model=None;
try:
  v2w_model = gensim. models. KeyedVectors.load("./w2vecmodel.mod")
  print("Loaded w2v model")
except:
  v2w_model= api.load('word2vec-google-news-300')
  v2w_model.save("./w2vecmodel.mod")
  print("Saved v2w model")
  w2vec_embedding_size=len (v2w_model['computer']); 

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Saved v2w model


In [16]:
#function to create phrase vectors from word vectors by simply suming up all the word vectors 
def getWordVec (word,model):
  samp=model['computer'];
  vec=[0]*len(samp);
  try:
        vec=model[word];
  except:
        vec=[0]*len(samp);
  return (vec)

def getPhraseEmbedding(phrase,embeddingmodel):
  
  samp=getWordVec('computer', embeddingmodel);
  vec=numpy.array([0]*len(samp));
  den=0;
  for word in phrase.split():
    den=den+1;
    vec=vec+numpy.array(getWordVec(word,embeddingmodel));
  
  return vec.reshape(1,-1)

In [41]:
#user query - simple
questn_orig='how can I pay?'

question=clean_sentence(questn_orig,stopwords=False)

In [42]:
#retrieving query based on simple user query  
cleaned_sentences=get_cleaned_sentences(df,stopwords=False)

sent_embeddings=[];
for sent in cleaned_sentences:
  sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.7358864746521492 do i have to register to shop at luluandsky com yes how do i register
1 0.5929838754323429 do i have to disclose my email  phone number for registration
2 0.4394731577608166 is my personal information secure at luluandsky com
3 0.45131341677424525 are there any charges for registration there are no charges for registration on luluandskycom will my registration expire
4 0.738082344317912 how do i get a new password
5 0.6096222995612963 can i edit my personal information
6 0.6890028645282527 how do i unsubscribe from your services
7 0.46066957497693345 is it possible
8 0.7487027689700474 how do i get in touch with luluandsky com
9 0.6395698974965346 what type of products do you sell online
10 0.7595452454625273 how do i know my size
11 0.864578815458327 how do i make a payment
12 0.6070836776685202 what happens if my creditdebit card has been compromised while making a payment online
13 0.5573714018595958 what happens if merchandise ordered is out of stock
14 0.66894

In [35]:
#user query - complex
questn_orig='how many days will it take for my order to get delivered?'
question=clean_sentence(questn_orig,stopwords=False)

In [43]:
#retrieving query based on complex user query  
cleaned_sentences=get_cleaned_sentences(df,stopwords=False)

sent_embeddings=[];
for sent in cleaned_sentences:
  sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.7358864746521492 do i have to register to shop at luluandsky com yes how do i register
1 0.5929838754323429 do i have to disclose my email  phone number for registration
2 0.4394731577608166 is my personal information secure at luluandsky com
3 0.45131341677424525 are there any charges for registration there are no charges for registration on luluandskycom will my registration expire
4 0.738082344317912 how do i get a new password
5 0.6096222995612963 can i edit my personal information
6 0.6890028645282527 how do i unsubscribe from your services
7 0.46066957497693345 is it possible
8 0.7487027689700474 how do i get in touch with luluandsky com
9 0.6395698974965346 what type of products do you sell online
10 0.7595452454625273 how do i know my size
11 0.864578815458327 how do i make a payment
12 0.6070836776685202 what happens if my creditdebit card has been compromised while making a payment online
13 0.5573714018595958 what happens if merchandise ordered is out of stock
14 0.66894

# BERT Training

In [26]:
#installing the bert serving client
!pip install bert-serving-client

!pip install -U bert-serving-server[http]

Requirement already up-to-date: bert-serving-server[http] in /usr/local/lib/python3.7/dist-packages (1.10.0)


In [27]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

!unzip uncased_L-12_H-768_A-12.zip

%tensorflow_version 1.x

!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

--2021-05-18 10:17:25--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M  76.3MB/s    in 5.5s    

2021-05-18 10:17:31 (71.1 MB/s) - ‘uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
replace uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001 

In [28]:
#importing bert client
from bert_serving.client import BertClient

bc = BertClient()

In [39]:
#user query - simple
questn_orig='how can I pay?'

question=clean_sentence(questn_orig,stopwords=False)

In [40]:
#retrieving query based on simple user query  
cleaned_sentences=get_cleaned_sentences(df,stopwords=False)

sent_bertphrase_embeddings=[];
for sent in cleaned_sentences:
  sent_bertphrase_embeddings.append(bc.encode([sent]));

question_embedding=bc.encode([question]);

retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df, cleaned_sentences);

0 0.7269781 do i have to register to shop at luluandsky com yes how do i register
1 0.7141696 do i have to disclose my email  phone number for registration
2 0.602797 is my personal information secure at luluandsky com
3 0.6438073 are there any charges for registration there are no charges for registration on luluandskycom will my registration expire
4 0.77977836 how do i get a new password
5 0.63833493 can i edit my personal information
6 0.70818913 how do i unsubscribe from your services
7 0.68586206 is it possible
8 0.73810434 how do i get in touch with luluandsky com
9 0.62330604 what type of products do you sell online
10 0.81619835 how do i know my size
11 0.8684416 how do i make a payment
12 0.66340995 what happens if my creditdebit card has been compromised while making a payment online
13 0.6429103 what happens if merchandise ordered is out of stock
14 0.6477229 is there a limit to the quantity that i can order
15 0.6835697 can i cancel an order
16 0.6625334 can i change the s

In [37]:
#user query - complex
questn_orig='how many days will it take for my order to get delivered?'

question=clean_sentence(questn_orig,stopwords=False)

In [38]:
#retrieving query based on complex user query  
cleaned_sentences=get_cleaned_sentences(df,stopwords=False)

sent_bertphrase_embeddings=[];
for sent in cleaned_sentences:
  sent_bertphrase_embeddings.append(bc.encode([sent]));

question_embedding=bc.encode([question]);

retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df, cleaned_sentences);

0 0.68481755 do i have to register to shop at luluandsky com yes how do i register
1 0.71404827 do i have to disclose my email  phone number for registration
2 0.59847045 is my personal information secure at luluandsky com
3 0.64724797 are there any charges for registration there are no charges for registration on luluandskycom will my registration expire
4 0.71004295 how do i get a new password
5 0.5795652 can i edit my personal information
6 0.5861496 how do i unsubscribe from your services
7 0.60096073 is it possible
8 0.7062543 how do i get in touch with luluandsky com
9 0.6537336 what type of products do you sell online
10 0.6628356 how do i know my size
11 0.7146776 how do i make a payment
12 0.69969475 what happens if my creditdebit card has been compromised while making a payment online
13 0.6629613 what happens if merchandise ordered is out of stock
14 0.6119648 is there a limit to the quantity that i can order
15 0.698671 can i cancel an order
16 0.7168655 can i change the sh